In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [3]:
len(purchase_data["SN"].value_counts())

576

## Purchasing Analysis (Total)

In [4]:
pd.DataFrame([[
    len(purchase_data["Item Name"].value_counts()),
    round(purchase_data["Price"].mean(),2),
    len(purchase_data),
    round(purchase_data["Price"].sum(),2)]],
    columns=["Unique Items", "Average Price", "Purchases", "Revenue"])

,Unique Items,Average Price,Purchases,Revenue
0,179,3.05,780,2379.77


## Gender Demographics

In [5]:
short_data = purchase_data[["SN","Gender"]]
short_data = short_data.drop_duplicates()

gender_df = pd.DataFrame(short_data["Gender"].value_counts())
gender_df["Percent"] = round(gender_df["Gender"] / gender_df["Gender"].sum(),4)
gender_df = gender_df.rename(columns={"Gender":"Total Count","Percent":"Percent of Players"})
gender_df.style.format({"Percent of Players":"{:.2%}"})

,Total Count,Percent of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [6]:
gender_analysis_df = gender_df
gender_analysis_df["Purchase Count"] = purchase_data.groupby("Gender")["Price"].count()
gender_analysis_df["Average Purchase Price"] = purchase_data.groupby("Gender")["Price"].mean()
gender_analysis_df["Total Purchase Value"] = purchase_data.groupby("Gender")["Price"].sum()
gender_analysis_df["Avg Total Purchase per Person"] = purchase_data.groupby("Gender")["Price"].sum() / gender_df["Total Count"]
gender_analysis_df

,Total Count,Percent of Players,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Male,484,0.8403,652,3.017853,1967.64,4.065372
Female,81,0.1406,113,3.203009,361.94,4.468395
Other / Non-Disclosed,11,0.0191,15,3.346000,50.19,4.562727


In [7]:
del gender_analysis_df["Total Count"]
del gender_analysis_df["Percent of Players"]

In [8]:
gender_analysis_df.style.format({"Average Purchase Price":"${:20,.2f}","Total Purchase Value":"${:20,.2f}","Avg Total Purchase per Person":"${:20,.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Male,652,$ 3.02,"$ 1,967.64",$ 4.07
Female,113,$ 3.20,$ 361.94,$ 4.47
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$ 4.56


## Age Demographics

In [9]:
bins = [0,9.5,14.5,19.5,24.5,29.5,34.5,39.5,100]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
age_df = pd.DataFrame(purchase_data[["Age","SN"]])
age_df["Range"] = pd.cut(age_df["Age"],bins,labels=group_names,include_lowest=True)
age_range_df = age_df.set_index("Range")
age_range_df = age_range_df.drop_duplicates()
age_range_df = age_range_df.groupby("Range").count()
del age_range_df["SN"]
age_range_df["Percentage of Players"] = age_range_df["Age"] / age_range_df["Age"].sum()
age_range_df = age_range_df.rename(columns={"Age":"Total Count"})
age_range_df.style.format({"Percentage of Players":"{:.2%}"})

,Total Count,Percentage of Players
Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [10]:
purch_by_age = purchase_data
purch_by_age["Range"] = pd.cut(purch_by_age["Age"],bins,labels=group_names,include_lowest=True)
purch_by_age = purch_by_age[["Range","Price"]]
age_purch_df = pd.merge(purch_by_age.groupby("Range").count(),purch_by_age.groupby("Range").mean(),on="Range")
age_purch_df = pd.merge(age_purch_df,purch_by_age.groupby("Range").sum(),on="Range")
age_purch_df = pd.merge(age_purch_df,age_range_df,on="Range")
age_purch_df["Avg Purch per Person"] = age_purch_df["Price"] / age_purch_df["Total Count"]
age_purch_df = age_purch_df.rename(columns={"Price_x":"Purches","Price_y":"Avg Purchase","Price":"Total Value"})
del age_purch_df["Total Count"]
del age_purch_df["Percentage of Players"]
age_purch_df.style.format({"Avg Purchase":"${:20,.2f}","Total Value":"${:20,.2f}","Avg Purch per Person":"${:20,.2f}"})

,Purches,Avg Purchase,Total Value,Avg Purch per Person
Range,,,,
<10,23,$ 3.35,$ 77.13,$ 4.54
10-14,28,$ 2.96,$ 82.78,$ 3.76
15-19,136,$ 3.04,$ 412.89,$ 3.86
20-24,365,$ 3.05,"$ 1,114.06",$ 4.32
25-29,101,$ 2.90,$ 293.00,$ 3.81
30-34,73,$ 2.93,$ 214.00,$ 4.12
35-39,41,$ 3.60,$ 147.67,$ 4.76
40+,13,$ 2.94,$ 38.24,$ 3.19


## Top Spenders

In [11]:
SN_analysis_df = pd.merge(purchase_data.groupby("SN").count(),purchase_data.groupby("SN").mean(),on="SN")
SN_analysis_df = pd.merge(SN_analysis_df,purchase_data.groupby("SN").sum(),on="SN")
SN_analysis_df = SN_analysis_df[["Age_x","Price_y","Price"]]
SN_analysis_df = SN_analysis_df.rename(columns={"Age_x":"Purchases","Price_y":"Avg Price","Price":"Total Value"})
SN_analysis_df = SN_analysis_df.sort_values("Purchases",ascending=False)
SN_analysis_df.head().style.format({"Avg Price":"${:20,.2f}","Total Value":"${:20,.2f}"})


,Purchases,Avg Price,Total Value
SN,,,
Lisosia93,5,$ 3.79,$ 18.96
Iral74,4,$ 3.40,$ 13.62
Idastidru52,4,$ 3.86,$ 15.45
Asur53,3,$ 2.48,$ 7.44
Inguron55,3,$ 3.70,$ 11.11


## Most Popular Items

In [12]:
most_popular_df = purchase_data[["Item ID","Item Name","Price"]]
pop_stats_df = pd.merge(most_popular_df.groupby(["Item ID","Item Name"]).mean(),most_popular_df.groupby(["Item ID","Item Name"]).sum(),on=["Item ID","Item Name"])
pop_stats_df = pd.merge(pop_stats_df,most_popular_df.groupby(["Item ID","Item Name"]).count(),on=["Item ID","Item Name"])
pop_stats_df = pop_stats_df.rename(columns={"Price_x":"Item Price","Price_y":"Total Value","Price":"Count"})
pop_stats_df = pop_stats_df.sort_values("Count",ascending=False)
pop_stats_df.head().style.format({"Item Price":"${:20,.2f}","Total Value":"${:20,.2f}"})


,,Item Price,Total Value,Count
Item ID,Item Name,,,
92,Final Critic,$ 4.61,$ 59.99,13
178,"Oathbreaker, Last Hope of the Breaking Storm",$ 4.23,$ 50.76,12
145,Fiery Glass Crusader,$ 4.58,$ 41.22,9
132,Persuasion,$ 3.22,$ 28.99,9
108,"Extraction, Quickblade Of Trembling Hands",$ 3.53,$ 31.77,9


## Most Profitable Items

In [13]:
pop_stats_df = pop_stats_df.sort_values("Total Value",ascending=False)
pop_stats_df.head().style.format({"Item Price":"${:20,.2f}","Total Value":"${:20,.2f}"})


,,Item Price,Total Value,Count
Item ID,Item Name,,,
92,Final Critic,$ 4.61,$ 59.99,13
178,"Oathbreaker, Last Hope of the Breaking Storm",$ 4.23,$ 50.76,12
82,Nirvana,$ 4.90,$ 44.10,9
145,Fiery Glass Crusader,$ 4.58,$ 41.22,9
103,Singed Scalpel,$ 4.35,$ 34.80,8
